In [1]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import time 
import transformers
import torch
from langchain.prompts import PromptTemplate
from langchain.callbacks import StdOutCallbackHandler

d:\development\aiml\local-hf-model-demo\local-hf-model-demo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name_path = "../../model/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name_path)
print("Tokenizer loaded")
print(tokenizer.tokenize("Hello World"))
model = AutoModelForCausalLM.from_pretrained(model_name_path, local_files_only=True) 
print("Model Loaded")

Tokenizer loaded
['▁Hello', '▁World']
Model Loaded


In [4]:
hf_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    do_sample=True,
    temperature=0.1,
    max_new_tokens=512,
    repetition_penalty=1.1
)

hfpipeline = HuggingFacePipeline(pipeline=hf_pipeline)
template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template=template)

chain  = prompt | hfpipeline 

handler = StdOutCallbackHandler()

question = "Who is the most corrupted politician of India"
print(chain.invoke({"question": question},{"callbacks": [handler]}))





> Entering new RunnableSequence chain...


> Entering new PromptTemplate chain...

> Finished chain.

> Finished chain.
Question: Who is the most corrupted politician of India
Answer: Let's think step by step.
1. Narendra Modi: He is the current Prime Minister of India and has been in power since 2014.
2. Lal Krishna Advani: He was the former Prime Minister of India from 1998 to 2004.
3. Rajnath Singh: He is the current Defence Minister of India and has served as the Home Minister of India twice.
4. Arun Jaitley: He is the former Finance Minister of India and has served as the Leader of Opposition in the Lok Sabha (Lower House of Parliament) for several times.
5. Sushma Swaraj: She is a former Union Minister of External Affairs of India and has served as the Foreign Minister of India multiple times.
6. Amit Shah: He is the current Bharatiya Janata Party (BJP) President and has served as the Home Minister of India multiple times.
7. Mukhtar Abbas Naqvi: He is the former Union Minister

In [5]:
from langchain.pydantic_v1 import BaseModel, Field, validator
import random

class DocumentComplete(BaseModel):
    """Checks the document specified via query and validates whether all the fields are populated for SOFT drawdown."""
    query: str = Field(description="customer drawdown request document.")
    
def isDocumentCompletedProperly(query):
    """Given a SOFR drawdown request document; return whether it is complete or incomplete."""
    
    bool_value  = bool(randon.getrandbits(1))
    
    if (bool_value):
        return {"response": "Document is filled out properly."}
    else:
        return {"response": "Document is not filled out properly."}


In [6]:
import random

class Customer(BaseModel):
    """Has everything about customer and can use that to query whether customer exists."""
    query: str = Field(description="customer name to search within master data to see whether they exists.")
    
def isCustomerExists(query):
    """returns whether a customer exists in our master data."""
    
    bool_value  = bool(randon.getrandbits(1))
    
    if (bool_value):
        return {"response": "Customer exists in our master data."}
    else:
        return {"response": "Customer does not exists in our master data."}

In [9]:
from langchain.agents import initialize_agent
from langchain.agents import Tool

is_document_complete_tool = Tool (
    name = "is_document_complete",
    func = isDocumentCompletedProperly,
    description = "Returns if customer supplied SOFR drawdown request document has all required fields or not.",
    args_schema = DocumentComplete
)



In [11]:
from langchain.agents import initialize_agent
from langchain.agents import Tool

is_customer_exists_tool = Tool (
    name = "is_customer_exists",
    func = isCustomerExists,
    description = "Returns whether the customer exists in our master data.",
    args_schema = Customer
)


In [12]:
tools = [
    is_document_complete_tool,
    is_customer_exists_tool
]

In [16]:
PREFIX = "<<SYS>> you are smart that selects a function from list of functions based on use queries. Run only one function tool at a time or in one query.<<SYS>>\n"

In [17]:
agent  = initialize_agent(
    tools,
    hfpipeline,
    agent = "zero-shot-react-description",
    verbose=True,
    max_iteration = 4,
    handle_parsing_errors=True,
    agent_kwargs={
        'prefix': PREFIX
    }
)

In [18]:
print(agent.agent.llm_chain.prompt.template)

<<SYS>> you are smart that selects a function from list of functions based on use queries. Run only one function tool at a time or in one query.<<SYS>>


is_document_complete: Returns if customer supplied SOFR drawdown request document has all required fields or not.
is_customer_exists: Returns whether the customer exists in our master data.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [is_document_complete, is_customer_exists]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [19]:
response = agent.run("Is the drawdown document for GME exists complete in context of SOFR drawdown?")
print(response)

d:\development\aiml\local-hf-model-demo\local-hf-model-demo\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: <<SYS>> you are smart that selects a function from list of functions based on use queries. Run only one function tool at a time or in one query.<<SYS>>


is_document_complete: Returns if customer supplied SOFR drawdown request document has all required fields or not.
is_customer_exists: Returns whether the customer exists in our master data.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [is_document_complete, is_customer_exists]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Is the drawdown document for GME exists complete in context of SOFR

Token indices sequence length is longer than the specified maximum sequence length for this model (2529 > 2048). Running this sequence through the model will result in indexing errors


Parsing LLM output produced both a final answer and a parse-able action:: <<SYS>> you are smart that selects a function from list of functions based on use queries. Run only one function tool at a time or in one query.<<SYS>>


is_document_complete: Returns if customer supplied SOFR drawdown request document has all required fields or not.
is_customer_exists: Returns whether the customer exists in our master data.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [is_document_complete, is_customer_exists]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Is the drawdown document for GME exists complete in context of SOFR drawdown?
Thought:Parsing LLM output pr

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Parsing LLM output produced both a final answer and a parse-able action:: <<SYS>> you are smart that selects a function from list of functions based on use queries. Run only one function tool at a time or in one query.<<SYS>>


is_document_complete: Returns if customer supplied SOFR drawdown request document has all required fields or not.
is_customer_exists: Returns whether the customer exists in our master data.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [is_document_complete, is_customer_exists]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Is the drawdown document for GME exists complete in context of SOFR drawdown?
Thought:Parsing LLM output pr